In [8]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import json
import datetime
load_dotenv()
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [2]:
def extractData(symbol, interval, range):
    url = "https://yh-finance.p.rapidapi.com/stock/v3/get-chart"
    querystring = {  "region":"US", "lang":"en", "symbol": symbol, "interval":interval,
                   "range": range, "events":"capitalGain,div,split"}
    headers = {"X-RapidAPI-Key": os.environ.get("RAPIDAPI_KEY"),
    "X-RapidAPI-Host": os.environ.get("RAPIDAPI_HOST")}
    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.ok:
        data = json.loads(response.content)
        return data["chart"]["result"][0]
    return None

In [3]:
response = extractData("V", "1m", "1d")
print(response)

{'meta': {'currency': 'USD', 'symbol': 'V', 'exchangeName': 'NYQ', 'instrumentType': 'EQUITY', 'firstTradeDate': 1205933400, 'regularMarketTime': 1677877202, 'gmtoffset': -18000, 'timezone': 'EST', 'exchangeTimezoneName': 'America/New_York', 'regularMarketPrice': 223.77, 'chartPreviousClose': 219.06, 'previousClose': 219.06, 'scale': 3, 'priceHint': 2, 'currentTradingPeriod': {'pre': {'timezone': 'EST', 'end': 1677853800, 'start': 1677834000, 'gmtoffset': -18000}, 'regular': {'timezone': 'EST', 'end': 1677877200, 'start': 1677853800, 'gmtoffset': -18000}, 'post': {'timezone': 'EST', 'end': 1677891600, 'start': 1677877200, 'gmtoffset': -18000}}, 'tradingPeriods': [[{'timezone': 'EST', 'end': 1677877200, 'start': 1677853800, 'gmtoffset': -18000}]], 'dataGranularity': '1m', 'range': '1d', 'validRanges': ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']}, 'timestamp': [1677853800, 1677853860, 1677853920, 1677853980, 1677854040, 1677854100, 1677854160, 1677854220, 167

In [87]:
def round_decimal(numbers):
    for i in range(len(numbers)):
        numbers[i] = round(numbers[i],2)
 

In [88]:
def create_df_metadata(metadata_response):
    trading_periods = metadata_response["tradingPeriods"]
    start_time = datetime.datetime.fromtimestamp(trading_periods[0][0]['start']).strftime("%H:%M")
    end_time = datetime.datetime.fromtimestamp(trading_periods[0][0]['end']).strftime("%H:%M")
    period = start_time + " - " + end_time

    important_columns = {"currency" : metadata_response["currency"], "symbol": metadata_response["symbol"], 
                         "instrumentType":metadata_response["instrumentType"],
                         "regularMarketPrice":metadata_response["regularMarketPrice"],
                         "previousClose":metadata_response["previousClose"], "trade_period":period,
                         'timezone': metadata_response['timezone'], "range": metadata_response["range"],
                         "interval": metadata_response["dataGranularity"]}
    df_metadata = pd.DataFrame(important_columns, index =[0])

    return df_metadata

def create_df_timestamp(timestamp_response):
    timestamps = response["timestamp"]
    converted_timestamp = []
    for timestamp in timestamps:
        converted_timestamp.append(datetime.datetime.fromtimestamp(timestamp).strftime("%H:%M"))
    df_timestamp = pd.DataFrame({"timestamp": converted_timestamp})
    return df_timestamp


def create_df_indicators(indicators_response):
    indicators = response["indicators"]["quote"]
    indicators_dictionary = {"Volume": round_decimal(indicators[0]["volume"]),
                         "Open": round_decimal(indicators[0]["open"]),
                         "Close": round_decimal(indicators[0]["close"]),
                         "High": round_decimal(indicators[0]["high"]),
                         "Low": round_decimal(indicators[0]["low"])}

    df_indicators = pd.DataFrame(indicators_dictionary)

    return df_indicators

In [89]:
df_indicator = create_df_indicators(response["indicators"])
df_metadata = create_df_metadata(response["meta"])
df_timestamp = create_df_timestamp(response["timestamp"])
merged_timestamp_indicators = pd.concat([df_timestamp, df_indicator], axis=1)
print(merged_timestamp_indicators)

    timestamp  Volume    Open   Close    High     Low
0       09:30  114079  220.20  220.36  220.54  219.90
1       09:31   19257  220.33  220.34  220.45  220.10
2       09:32   12650  220.37  220.18  220.37  220.11
3       09:33   21735  220.20  220.24  220.38  220.12
4       09:34   34754  220.24  220.80  220.84  220.23
..        ...     ...     ...     ...     ...     ...
386     15:56   52870  223.94  223.84  224.01  223.82
387     15:57   46600  223.84  223.88  223.94  223.83
388     15:58   64575  223.88  223.80  223.89  223.62
389     15:59   79526  223.80  223.71  223.88  223.68
390     16:00       0  223.77  223.77  223.77  223.77

[391 rows x 6 columns]
